In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from datasets import Dataset
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch
import heapq
import sys
sys.path.append("/mnt/bn/data-tns-live-llm/leon/Cherry_LLM")
from niid_data.utils import process_sft_dataset

In [2]:
code_data = load_dataset("sahil2801/CodeAlpaca-20k")["train"]
fin_data = load_dataset("FinGPT/fingpt-sentiment-train")["train"]
med_data = load_dataset("medalpaca/medical_meadow_medical_flashcards")["train"]
general_data = load_dataset("tatsu-lab/alpaca")["train"]
math_data = load_dataset("TIGER-Lab/MathInstruct")["train"]

In [9]:
import random
random.seed(42)
client_data = process_sft_dataset("sahil2801/CodeAlpaca-20k",code_data)
iid_idxs = random.sample(range(len(client_data)), 1000)
base_data = client_data.select(iid_idxs)
clients_data = []
for i in range(10):
    clients_data.append(base_data.shard(10,i))

>> ===== After processing, Dataset sahil2801/CodeAlpaca-20k has 20022 examples. =====


In [4]:
import requests
import json

def get_gpt(prompt, format):
    data = {
        "model": "gpt-3.5-turbo-0125",
        "messages":  [
            {"role": "system", "content": f"You are a helpful assistant designed to output JSON. The format is: {format}"},
            {"role": "user", "content": prompt}
        ],
        "temperature": 1,
        "top_p": 1,
        "n": 1,
        "stream": False,
        "stop": None,
        "max_tokens": 512,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "user": None,
        "response_format": {"type": "json_object"}
    }
    headers = {'Content-Type': 'application/json', 'Caller': 'leon.kepler'}
    data = {k: v for k, v in data.items() if v is not None}
    data = json.dumps(data)
    url = f"https://swzkkd0h.us-east-fn.bytedance.net/gpt/openapi/online/v2/crawl"
    response = requests.post(url, data=data, headers=headers)
    return response.json()["choices"][0]["message"]["content"]

# 尝试 vllm json mode

In [71]:
from pydantic import BaseModel, constr

instruction_schema = """{
    "title": "Instruction Schema",
    "type": "object",
    "properties": {
        "Instruction": {
            "title": "instruction",
            "type": "string"
        }
    },
    "required": ["Instruction"]
}"""

response_schema = """{
    "title": "Response Schema",
    "type": "object",
    "properties": {
        "Response": {
            "title": "response",
            "type": "string"
        }
    },
    "required": ["Response"]
}"""

In [93]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="damn"
)

def get_vllm(prompt, schema):
  completion = client.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct/",
    messages=[
      {"role": "system", "content": f"You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": prompt},
    ],
    extra_body={"guided_json": schema}
  )
  return json.loads(completion.choices[0].message.content)

In [11]:
instructions = []
response_examples = []
for i in range(100):
    instructions.append(random.sample(clients_data[0]["instruction"],4))
    response_examples.append(clients_data[0].select(random.sample(range(len(clients_data[0])),2)))

In [12]:
import evaluate
rouge = evaluate.load('rouge')
def compute_sim(instruction):
    return rouge.compute(predictions=[instruction], references=[clients_data[0]["instruction"]],rouge_types=['rougeL'])["rougeL"]

In [67]:
instruction_prompt = """You are asked to come up with instructions. These instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions. Don't repeat instructions in examples. Here are some examples: Instruction 1: {} Instruction 2: {} Instruction 3: {} Instruction 4: {} Provide a new instruction below:"""

instruction_format = "{Instruction: instruction}"

In [73]:
def generate_instruction(prompt, instruction_format=instruction_format):
    cnt=0
    while cnt!=2:
        cnt+=1
        try:
            # instruction = json.loads(get_gpt(prompt,instruction_format))["Instruction"]
            instruction = get_vllm(prompt, instruction_schema)["Instruction"]
        except:
            print("generate instruction error")
            continue
        # if compute_sim(instruction)>0.7: 
        #     print(instruction)
        #     continue
        break
    return instruction

In [82]:
response_format="{Response: response}"

def generate_response(prompt, response_format=response_format):
    cnt=0
    while cnt!=2:
        cnt+=1
        # output = get_gpt(prompt,response_format)
        output = get_vllm(prompt,response_schema)
        try:
            # output = json.loads(output)["Response"]
            output = output["Response"]
            break
        except:
            print(f"gen response error output")
    return output

In [70]:
tmp = instruction_prompt.format(*instructions[0])
generate_instruction(tmp,instruction_format)

You are asked to come up with instructions. These instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions. Don't repeat instructions in examples. Here are some examples: Instruction 1: Write a for loop in JavaScript that prints numbers from 0 to 5. Instruction 2: Compare two objects and return 0 if they are equal, -1 if the first is less than the second, and 1 if the first is greater than the second. object1 = 7
object2 = 8 Instruction 3: Develop a code to sort a list in ascending or descending order. givenList: [5, -3, 2, 4, 0], sortOrder: "ascending" Instruction 4: Insert a node at the beginning of a linked list. [10 -> 15 -> 20 -> 25] Provide a new instruction below:


'Write a Python function that takes a string as input and returns the string with all vowels removed.'

In [90]:
response_format = """
Example 1: 
    Instruction: {}
    Response: {}
Example 2: 
    Instruction: {}
    Response: {}

Generate the response of this instruction, Instruction: {}
"""

In [18]:
def format_response_prompt(idx, instruction):
    response_seqs = []
    for data in response_examples[idx]:
        response_seqs.extend([data["instruction"],data["response"]])
    response_seqs.append(instruction)
    prompt = response_format.format(*response_seqs)
    return prompt

In [95]:
response_seqs = []
for data in response_examples[0]:
    response_seqs.extend([data["instruction"],data["response"]])
print(response_seqs)

['Write an algorithm that prints the first 10 Fibonacci numbers in descending order.', '# Initialize the first two numbers\na = 0\nb = 1\n\n# Print the first 10 Fibonacci numbers\nfor i in range(10):\n    # Swap the values\n    a, b = b, a + b\n    # Print the current Fibonacci number\n    print(b)\n# Reverse the list\nfor _ in range(9, 0, -1):\n    print(_)', 'Write a SQL statement to update the age of a customer with a specific ID. Customer_id = 24, Age = 30', 'UPDATE customers \nSET age = 30\nWHERE customer_id = 24;']


In [96]:
instruction = "Write a Python function that takes a string as input and returns the string with all vowels removed."
response_seqs.append(instruction)
tmp_prompt = response_format.format(*response_seqs)
tmp_prompt

'\nExample 1: \n    Instruction: Write an algorithm that prints the first 10 Fibonacci numbers in descending order.\n    Response: # Initialize the first two numbers\na = 0\nb = 1\n\n# Print the first 10 Fibonacci numbers\nfor i in range(10):\n    # Swap the values\n    a, b = b, a + b\n    # Print the current Fibonacci number\n    print(b)\n# Reverse the list\nfor _ in range(9, 0, -1):\n    print(_)\nExample 2: \n    Instruction: Write a SQL statement to update the age of a customer with a specific ID. Customer_id = 24, Age = 30\n    Response: UPDATE customers \nSET age = 30\nWHERE customer_id = 24;\n\nGenerate the response of this instruction, Instruction: Write a Python function that takes a string as input and returns the string with all vowels removed.\n'

In [94]:
generate_response(tmp_prompt)

"def remove_vowels(input_string):\\\\n    vowels = 'aeiouAEIOU'\\\\n    output_string = ''\\\\n    for char in input_string:\\\\n        if char not in vowels:\\\\n            output_string += char\\n    return output_string"

In [19]:
gen_instructions = []
gen_responses = []
for i in tqdm(range(50)):
    tmp = instruction_prompt.format(*instructions[i])
    instruction = generate_instruction(tmp,instruction_format)
    prompt = format_response_prompt(i,instruction)
    response = generate_response(prompt)
    gen_instructions.append(instruction)
    gen_responses.append(response)

100%|██████████| 50/50 [03:01<00:00,  3.62s/it]


In [20]:
df = pd.DataFrame({
    "instruction": gen_instructions,
    "response": gen_responses,
})
df.to_csv(f"./code.csv")

# 月之暗面！

In [10]:
import json
 
from openai import OpenAI
 
client = OpenAI(
    api_key="sk-Nl2yxo0Ic9ZJ0zFf3qWdypuoQ6Nd2xnsCp7wfojlW9OyOYmq", # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    base_url="https://api.moonshot.cn/v1",
)

# {
#     "response": "text",
# }
system_prompt = """
You are the intelligent customer service of Dark Side of the Moon (Kimi), and you are responsible for answering various questions raised by users. Please refer to the documentation to reply to the user's question. Your answer can be text, pictures, and links, and you can include text, pictures, and links in one reply.
Please use the following JSON format to output your response:
 
{
    "Task 1": "instruction",
    "Task 2": "instruction",
    "Task 3": "instruction",
    "Task 4": "instruction",
    "Task 5": "instruction",
}
"""
 
completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        {"role": "system",
         "content": "You are Kimi, an artificial intelligence assistant provided by Moonshot AI. You are better at conversation in Chinese and English. You will provide users with safe, helpful and accurate answers. At the same time, you will reject all answers involving terrorism, racism, pornographic violence, etc. Moonshot AI is a proper noun and cannot be translated into other languages."},
        {"role": "system", "content": system_prompt}, # <-- 将附带输出格式的 system prompt 提交给 Kimi
        {"role": "user", "content": prompt}
    ],
    temperature=0.3,
    response_format={"type": "json_object"}, # <-- 使用 response_format 参数指定输出格式为 json_object
)
 
content = json.loads(completion.choices[0].message.content)

In [11]:
print(content)

{'Task 1': "The sentiment of the news 'The goal is to secure the company's profitability and competitiveness' can be considered as positive, as it implies a forward-looking and optimistic approach to business growth.", 'Task 2': "The sentiment of the news about Apple's first foray into live sports broadcasting can be seen as moderately positive, as it indicates a new venture and potential for growth in a new market segment.", 'Task 3': 'The sentiment of the news about semiconductor stocks falling and nearing bear-market territory is moderately negative, as it suggests a downturn in the market and potential financial concerns for investors.', 'Task 4': "The sentiment of the tweet about Iberchem's record year and growth in revenue is positive, as it highlights successful business performance and expansion.", 'Task 5': 'The sentiment of the news about Costco being watched by Zacks.com users can be interpreted as mildly positive, as it suggests interest and potential positive attention tow

In [10]:
data_concated: Dataset = processed_data[0]
random.seed(42)
iid_idxs = random.sample(range(len(data_concated)), 1000)
base_data = data_concated.select(iid_idxs)
clients_data = []
for i in range(10):
    clients_data.append(base_data.shard(10,i))

data_concated = data_concated.select(list(set(range(len(data_concated)))-set(iid_idxs)))
print(len(data_concated))

75772


In [14]:
print(clients_data[0].column_names)

['instruction', 'response', 'label']


In [16]:
from datasets import load_dataset
import json

domain = "financial"
# 定义转换函数，将数据集中的一行转换为所需的JSONL格式
def convert_to_jsonl(i,row):
    return {
        "id": f"seed_task_{i}",
        "name": row['name'] if 'name' in row else domain,  # 假设每行都有'name'列，如果没有，则使用默认值
        "instruction": row['instruction'],
        "instances": [{"input": "", "output": row['response']}],
        "is_classification": False
    }

# 打开文件用于写入
with open('output.jsonl', 'w') as f:
    # 遍历数据集中的所有行
    for i,item in enumerate(clients_data[0]):  # 假设我们处理的是训练集
        # 转换为JSONL格式
        jsonl_line = convert_to_jsonl(i,item)
        # 将JSON对象转换为JSON字符串，并写入文件
        f.write(json.dumps(jsonl_line) + '\n')

print("转换完成，文件已保存为 output.jsonl")

转换完成，文件已保存为 output.jsonl
